## Reinforcement Learning Project

**Studente:** Alessandro Mattei
**Matricola:** 295441
**Email:** alessandro.mattei1@student.univaq.it


In [1]:
from agent import Agent
from state import State

from world import World

## Introduzione
### Il problema del cuoco


Consideriamo il caso in cui l'agente sia il vostro cuoco personale.

In particolare, l'agente (lo smiley sulla mappa) vuole cucinare la ricetta delle uova in base alle
indicazioni dell'utente (strapazzate o in un budino).

Per cucinare la ricetta desiderata, l'agente deve prima raccogliere gli strumenti necessari (il frullino
per le uova sulla mappa). Poi deve raggiungere i fornelli (la padella o il forno sulla mappa). Infine,
può cucinare.

Non che ci siano due speciali celle interconnesse (contrassegnate con 1a G) che permettono
all'agente di passare da un lato all'altro della mappa. Ma per farlo, l'agente deve esprimere la sua
volontà di andare dall'altra parte.

Le celle in (4, 2) e (9, 3) sono i cancelli speciali. Permettono all'agente di passare da un lato
all'altro della mappa. Queste due celle speciali sono collegate tra loro, ma l'agente deve esprimere
la sua volontà di andare dall'altra parte.

Dato che siete molto affamati, è fondamentale che l'agente cucini le uova secondo i vostri gusti
(strapazzate/pudding) il più velocemente possibile senza farvi aspettare più del necessario.

![Figura 1](imgs/figura_1.png "figura 1")

Figura 1: Un'istanza particolare del problema di cottura di Chef. L'obiettivo è che l'agente
attualmente situato nello stato (4, 3) abbia una politica che porti sempre a cucinare le uova nella
posizione (1, 4) o (8, 4). Le celle in (4, 2) e (9, 3) sono quelle del cancello speciale.

Consideriamo l'istanza mostrata nella Figura 1:
- Nella figura, l'agente si trova in (4, 3) (ma può iniziare in qualsiasi cella della griglia).
- L'agente ha bisogno di strumenti di cottura come il frullino per le uova in posizione (1, 3) e (8,3).
- Ci sono due obiettivi finali diversi, visualizzati come la padella in posizione (1, 4) e il forno in posizione (8, 4).
- Le celle in (4, 2) e (9, 3) sono i cancelli speciali. Permettono all'agente di passare da un lato all'afro della mappa. Queste due celle speciali sono collegate tra loro, ma l'agente deve esprimere la sua volontà di andare dall'altra parte.
- L'agente non può muoversi in diagonale.
- Le pareti sono rappresentate da linee nere spesse.
- L'agente non può muoversi attraverso i muri.
- Un episodio terminerà quando l'agente riuscirà a cucinare le uova strapazzate (vedi descrizione precedente).

## Analisi

Per semplificare il problema consideriamo separatamente gli stati in cui l'agente non hanno in mano lo sbattitore da quelli che ha lo sbattitore (situato in posizione **(3,1)** e **(3.8)** visibile nelle foto in basso). 
Per far ciò ho duplicato la griglia.

![Figura 2](imgs/figura_2.png "figura 2")

Figura 2: prima griglia che contiene solo gli sbattitori

Nella prima griglia (nella figura sopra), l'agente ha l'obiettivo di raggiungere uno dei due **sbattitori**.

![Figura 3](imgs/figura_3.png "figura 3")

Figura 3: seconda griglia che contiene solo le postazioni di cottura

![Figura 4](imgs/figura_4.png "figura 4")

Figura 4: griglia complessiva. Una volta che l'agente avrà preso uno sbattitore potrà andare nella griglia 2 per cucinare (come indicato dalle frecce).

Non appena l'agente esegue l'azione **take_whiskr** da una delle due posizioni contenenti uno **sbattitore**, esso viene portato nella seconda griglia seguendo le frecce mostrate nella **figura 4**:
- Se l'agente prende lo sbattitore in posizione **(3, 1)** nella griglia in basso, esso viene portato nella griglia in alto in posizione **(7, 1)**
- Se l'agente prende lo sbattitore in posizione **(3, 8)** nella griglia in basso, esso viene portato nella griglia in alto in posizione **(7, 8)**

Dopo aver raggiunto e **preso uno sbattitore**, l'agente deve raggiungere una delle due postazioni di cucina che sono nella griglia superiore (situati nelle posizioni **(8, 1)** e **(8, 8)** nella griglia in alto) per cucinare le uova (la postazione di cucina da raggiungere dipende dalla ricetta che l'agente deve cucinare)

L'agente dovrà percorrere entrambe le griglie per raggiungere l'obiettivo finale. Dalla prima griglia alla seconda griglia.

Nella seconda griglia (griglia situata in alto), l'agente ha l'obiettivo di raggiungere una delle due postazioni di cucina e di eseguire quindi l'azione **cook**:
- la postazione di cucina per il piatto **scrambled eggs** è situato in posizione **(8, 1)**
- la postazione di cucina per il piatto **pudding eggs** è situato in posizione **(8, 8)**

In entrambe le griglie sono situati dei **Gate** che hanno la funzione di trasportare l'agente da una stanza all'altra delle stessa griglia, l'agente quando si trova su uno di questi può usare il **Gate** tramite l'azione **go_rigth** o **go_left**:
- la postazione di **Gate** situato in posizione **(2, 4)** trasporta l'agente in posizione **(3, 9)** se effettua l'azione **go_right**
- la postazione di **Gate** situato in posizione **(3, 9)** trasporta l'agente in posizione **(2, 4)** se effettua l'azione **go_left**
- la postazione di **Gate** situato in posizione **(6, 4)** trasporta l'agente in posizione **(7, 9)** se effettua l'azione **go_right**
- la postazione di **Gate** situato in posizione **(7, 9)** trasporta l'agente in posizione **(6, 4)** se effettua l'azione **go_left**

#### Definiamo tutti i possibili stati

Ogni stato del sistema corrisponde a una posizione possibile che l'agente può assumere nella griglia.

Tutte le posizione della **colonna 5** che non possono essere fisicamente raggiunte non saranno presenti tra gli stati possibili del problema (posizioni impossibili).

Vediamo una rappresentazione in Tupla posizionale.

In [2]:
world = World()
world.print_states(True)

 (8, 1)  (8, 2)  (8, 3)  (8, 4)      (8, 6)  (8, 7)  (8, 8)  (8, 9) 
 (7, 1)  (7, 2)  (7, 3)  (7, 4)      (7, 6)  (7, 7)  (7, 8)  (7, 9) 
 (6, 1)  (6, 2)  (6, 3)  (6, 4)      (6, 6)  (6, 7)  (6, 8)  (6, 9) 
 (5, 1)  (5, 2)  (5, 3)  (5, 4)      (5, 6)  (5, 7)  (5, 8)  (5, 9) 


 (4, 1)  (4, 2)  (4, 3)  (4, 4)      (4, 6)  (4, 7)  (4, 8)  (4, 9) 
 (3, 1)  (3, 2)  (3, 3)  (3, 4)      (3, 6)  (3, 7)  (3, 8)  (3, 9) 
 (2, 1)  (2, 2)  (2, 3)  (2, 4)      (2, 6)  (2, 7)  (2, 8)  (2, 9) 
 (1, 1)  (1, 2)  (1, 3)  (1, 4)      (1, 6)  (1, 7)  (1, 8)  (1, 9) 


Vediamo una rappresentazione dei stati in versione numerica.

In [3]:
world.print_states(False)

 56  57  58  59      60  61  62  63 
 48  49  50  51      52  53  54  55 
 40  41  42  43      44  45  46  47 
 32  33  34  35      36  37  38  39 


 24  25  26  27      28  29  30  31 
 16  17  18  19      20  21  22  23 
 8   9   10  11      12  13  14  15 
 0   1   2   3       4   5   6   7  


Il numero totale di Stati è 64.

In [4]:
print(len(world.S))
world.S

64


[State{number: 0, position: (1, 1)},
 State{number: 1, position: (1, 2)},
 State{number: 2, position: (1, 3)},
 State{number: 3, position: (1, 4)},
 State{number: 4, position: (1, 6)},
 State{number: 5, position: (1, 7)},
 State{number: 6, position: (1, 8)},
 State{number: 7, position: (1, 9)},
 State{number: 8, position: (2, 1)},
 State{number: 9, position: (2, 2)},
 State{number: 10, position: (2, 3)},
 State{number: 11, position: (2, 4)},
 State{number: 12, position: (2, 6)},
 State{number: 13, position: (2, 7)},
 State{number: 14, position: (2, 8)},
 State{number: 15, position: (2, 9)},
 State{number: 16, position: (3, 1)},
 State{number: 17, position: (3, 2)},
 State{number: 18, position: (3, 3)},
 State{number: 19, position: (3, 4)},
 State{number: 20, position: (3, 6)},
 State{number: 21, position: (3, 7)},
 State{number: 22, position: (3, 8)},
 State{number: 23, position: (3, 9)},
 State{number: 24, position: (4, 1)},
 State{number: 25, position: (4, 2)},
 State{number: 26, pos

## Definizioni delle azioni Possibili

L'agente può effettuare le **azioni di movimento** (**left**, **right**, **up**, **down**) e inoltre può effettuare alcune azioni specifiche per specifichi stati:
- L'azione **take_whiskr** corrisponde all'azione di prendere lo sbattitore. A seguito della sua applicazione, l'agente viene portato nella seconda griglia.
- L'azione **go_right** corrisponde all'azione di utilizzo del Gate posizionato nella stanza di sinistra. A seguito della sua applicazione, l'agente viene portato nella seconda stanza dalle prima stanza.
- L'azione **go_left** corrisponde all'azione di utilizzo del Gate posizionato nella stanza di destra. A seguito della sua applicazione, l'agente viene portato nella prima stanza dalla seconda stanza.
- L'azione **cook** corrisponde all'azione di cucinare le uova. In questo caso, l'applicazione di questa azione porta l'agente a rimanere fermo (absorbing state). L'esecuzione di questa azione rappresenta l'obiettivo finale dell'agente

L'insieme **A** è il seguente:
- azione 0: **left ←**
- azione 1: **right →**
- azione 2: **up ↑**
- azione 3: **down ↓**
- azione 4: **take_whiskr T**
- azione 5: **go_right R**
- azione 6: **go_left L**
- azione 7: **cook C**

In [5]:
world.A

[Action{name: left, number: 0, function: action_left, ascii: ←},
 Action{name: right, number: 1, function: action_right, ascii: →},
 Action{name: up, number: 2, function: action_up, ascii: ↑},
 Action{name: down, number: 3, function: action_down, ascii: ↓},
 Action{name: take_whiskr, number: 4, function: action_take_whiskr, ascii: T},
 Action{name: go_right, number: 5, function: action_go_right, ascii: R},
 Action{name: go_left, number: 6, function: action_go_left, ascii: L},
 Action{name: cook, number: 7, function: action_cook, ascii: C}]

Costruiamo ora una mappa delle azioni possibili per ogni stato.
In questo caso modelliamo questa mappa per permettere entrambe le ricette contemporaneamente, ma in seguito se decidiamo una ricetta elimineremo l'azione di cucinare in funzione della ricetta:
-  Se si decide di cucinare **scrambled eggs (8, 1)** allora l'azione **cook** allo stato **62 (8,8)** verrà eliminata dalla mappa
-  Se si decide di cucinare **pudding eggs (8,8)** allora l'azione **cook** allo stato **56 (8,1)** verrà eliminata dalla mappa

Un'altra alternativa alla rimozione dell'azione **cook** è quella di impostare una ricompensa estremamente negativa e svantaggiosa, così da non farla prediligere rispetto alle altre azioni possibili.

In [6]:
world.am

{0: [1],
 1: [0, 1],
 2: [0, 1],
 3: [0, 2],
 4: [1, 2],
 5: [0, 1, 2],
 6: [0, 1],
 7: [0, 2],
 8: [1, 2],
 9: [0, 1],
 10: [0, 1],
 11: [0, 2, 3, 5],
 12: [1, 2, 3],
 13: [0, 2, 3],
 14: [1, 2],
 15: [0, 2, 3],
 16: [3, 4],
 17: [1, 2],
 18: [0, 1, 2],
 19: [0, 2, 3],
 20: [1, 2, 3],
 21: [0, 2, 3],
 22: [1, 3, 4],
 23: [0, 3, 6],
 24: [1],
 25: [0, 1, 3],
 26: [0, 1, 3],
 27: [0, 3],
 28: [1, 3],
 29: [0, 1, 3],
 30: [0, 1],
 31: [0],
 32: [1],
 33: [0, 1],
 34: [0, 1],
 35: [0, 2],
 36: [1, 2],
 37: [0, 1, 2],
 38: [0, 1],
 39: [0, 2],
 40: [1, 2],
 41: [0, 1],
 42: [0, 1],
 43: [0, 2, 3, 5],
 44: [1, 2, 3],
 45: [0, 2, 3],
 46: [1, 2],
 47: [0, 2, 3],
 48: [3],
 49: [1, 2],
 50: [0, 1, 2],
 51: [0, 2, 3],
 52: [1, 2, 3],
 53: [0, 2, 3],
 54: [1, 3],
 55: [0, 3, 6],
 56: [1, 7],
 57: [0, 1, 3],
 58: [0, 1, 3],
 59: [0, 3],
 60: [1, 3],
 61: [0, 1, 3],
 62: [0, 1, 7],
 63: [0]}

## Creazione insieme di Transizioni P 
### d) Report the transition function $P$ for an state $s$ and action $a$ in a tabular format.

Creiamo la matrice a 3 dimensioni di transizione $P[A, S, S']$:
- $A$: sono le Azioni
- $S$: sono gli stati sorgente
- $S'$: sono gli stati di destinazione

Modelliamo il problema assumendo che non ci sono effetti imprevisti a seguito dell'applicazione delle azioni:

da uno stato **sorgente** applicando l'**azione**, abbiamo soltanto **uno stato destinazione** (stato raggiungibile con probabilità **1**)

![Figura 5](imgs/figura_5.png "figura 5")
Figura 5: Esempio di comportamento dell'agente a seguito dell'applicazione dell'azione **up**

In [7]:
world.P

array([[[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

Costruiamo le varie matrici di transizione $P$ per ogni azione specificata.

Stampiamo una tabella che descrive una matrice $S X S'$ per l'azione scelta.
$P[left, S, S']$

In [8]:
# transition_left = world.print_transitions_matrix(action='left')
# transition_left.to_csv("transitions/transition_left.csv")
# transition_left

<p>
$P[right, S, S']$
</p>

In [9]:
# transition_right = world.print_transitions_matrix(action='right')
# transition_right.to_csv("transitions/transition_right.csv")
# transition_right

<p>$P[up, S, S']$</p>

In [10]:
# transition_up = world.print_transitions_matrix(action='up')
# transition_up.to_csv("transitions/transition_up.csv")
# transition_up

<p>$P[down, S, S']$</p>

In [11]:
# transition_down = world.print_transitions_matrix(action="down")
# transition_down.to_csv("transitions/transition_down.csv")
# transition_down

<p>$P[take_whiskr, S, S']$</p>

In [12]:
# transition_take_whiskr = world.print_transitions_matrix(action='take_whiskr')
# transition_take_whiskr.to_csv("transitions/transition_take_whiskr.csv")
# transition_take_whiskr

<p>$P[go_right, S, S']$</p>

In [13]:
# transition_go_right = world.print_transitions_matrix(action='go_right')
# transition_go_right.to_csv("transitions/transition_go_right.csv")
# transition_go_right

<p>$P[go_left, S, S']$</p>

In [14]:
# transition_go_left = world.print_transitions_matrix(action='go_left')
# transition_go_left.to_csv("transitions/transition_go_left.csv")
# transition_go_left

<p>$P[cook, S, S']$</p>

In [15]:
# transition_cook = world.print_transitions_matrix(action='cook')
# transition_cook.to_csv("transitions/transition_cook.csv")
# transition_cook

## Creazione Insieme di reward $R$
### e) Describe a reward function $R : S × A × S$ and a value of $γ$ that will lead to an optimal policy

Il nostro obbiettivo è cucinare il piatto **pudding eggs**.

Un possibile approccio alla scelta dei rewards da assegnare all'agente per guidarlo nella risoluzione del problema è il seguente:
- Premio per l'obbiettivo $+100$ cucinare il piatto **pudding eggs** allo stato 62
- Premio per l'obbiettivo intermedio $+50$ prendere uno sbattitore posizionato allo stato **16** o **22**
- Penalizzazione per un azione non valida $-1$
- nei casi rimanenti il Premio sarà $0$

In [16]:
# caso cook (goal)
print(world.R[62, 7, 62])
# caso cook sbagliato
print(world.R[56, 7, 56])
# caso take_whiskr
print(world.R[16, 4, 48])
# caso take_whiskr
print(world.R[22, 4, 54])
# caso non valido
# lo stato 22 ha le seguenti azioni: [1, 3, 4]
world.R[22, 2, 22]

1000.0
-10.0
500.0
500.0


-10.0

## implementazione scenario Model free

### h) Now, considering the problem as a model-free scenario, provide a program (written in Python, possibly based on the labs) that can compute the optimal policy for this world by solely considering the pudding eggs scenario. Draw the computed policy in the grid by putting the optimal action in each cell. If multiple actions are possible, include the probability of each arrow. There may be multiple optimal policies; pick one to show it. Note that the model is not available for computation but must be encoded to be used as the "real-world" environment.

In questo caso, non sarà possibile accedere al **transition model** e al **reward model** del problema.

Per la specifica **Model-free**, consideriamo soltanto lo scenario **pudding eggs**.

Definiamo una funzione di policy iniziale insieme a una funzione di generazione di episodi. Useremo e adatteremo il codice sviluppato durante il laboratorio.

In [17]:
def simple_policy(s: State):
    return world.am[s.number][0]

Definiamo una funzione che ci calcola una policy ottima, utilizzeremo la funzione vista in laboratorio.

In [18]:
#generate_episode()
agent = Agent(simple_policy)
#print(agent.generate_episode(50, return_actions=True))
agent.improve_policy(10)

Iterazione: 0/10
Iterazione: 1/10
Iterazione: 2/10
Iterazione: 3/10
Iterazione: 4/10
Iterazione: 5/10
Iterazione: 6/10
Iterazione: 7/10
Iterazione: 8/10
Iterazione: 9/10
{(19, 'left'): -2.804838774119153e-29, (19, 'right'): -10.0, (19, 'up'): 0.0, (19, 'down'): 0.0, (19, 'take_whiskr'): -10.0, (19, 'go_right'): -10.0, (19, 'go_left'): -10.0, (19, 'cook'): -10.0, (18, 'left'): -0.09001406469760931, (18, 'right'): -1.5777218104420236e-29, (18, 'up'): -1.5777218104420236e-29, (18, 'down'): -10.0, (18, 'take_whiskr'): -10.0, (18, 'go_right'): -10.0, (18, 'go_left'): -10.0, (18, 'cook'): -10.0, (17, 'left'): -10.0, (17, 'right'): -0.04562303963501584, (17, 'up'): 0.0, (17, 'down'): -10.0, (17, 'take_whiskr'): -10.0, (17, 'go_right'): -10.0, (17, 'go_left'): -10.0, (17, 'cook'): -10.0, (27, 'left'): -1.7530242338244708e-30, (27, 'right'): -10.0, (27, 'up'): -10.0, (27, 'down'): -1.5777218104420236e-29, (27, 'take_whiskr'): -10.0, (27, 'go_right'): -10.0, (27, 'go_left'): -10.0, (27, 'cook'):

{(19, 'left'): 0,
 (19, 'right'): 0,
 (19, 'up'): 1,
 (19, 'down'): 0,
 (19, 'take_whiskr'): 0,
 (19, 'go_right'): 0,
 (19, 'go_left'): 0,
 (19, 'cook'): 0,
 (18, 'left'): 0,
 (18, 'right'): 1,
 (18, 'up'): 0,
 (18, 'down'): 0,
 (18, 'take_whiskr'): 0,
 (18, 'go_right'): 0,
 (18, 'go_left'): 0,
 (18, 'cook'): 0,
 (17, 'left'): 0,
 (17, 'right'): 0,
 (17, 'up'): 1,
 (17, 'down'): 0,
 (17, 'take_whiskr'): 0,
 (17, 'go_right'): 0,
 (17, 'go_left'): 0,
 (17, 'cook'): 0,
 (27, 'left'): 1,
 (27, 'right'): 0,
 (27, 'up'): 0,
 (27, 'down'): 0,
 (27, 'take_whiskr'): 0,
 (27, 'go_right'): 0,
 (27, 'go_left'): 0,
 (27, 'cook'): 0,
 (26, 'left'): 0,
 (26, 'right'): 1,
 (26, 'up'): 0,
 (26, 'down'): 0,
 (26, 'take_whiskr'): 0,
 (26, 'go_right'): 0,
 (26, 'go_left'): 0,
 (26, 'cook'): 0,
 (25, 'left'): 0,
 (25, 'right'): 1,
 (25, 'up'): 0,
 (25, 'down'): 0,
 (25, 'take_whiskr'): 0,
 (25, 'go_right'): 0,
 (25, 'go_left'): 0,
 (25, 'cook'): 0,
 (24, 'left'): 0,
 (24, 'right'): 1,
 (24, 'up'): 0,
 (24,

In [19]:
agent.easy_policy

{19: 'up',
 18: 'right',
 17: 'up',
 27: 'left',
 26: 'right',
 25: 'right',
 24: 'right',
 11: 'go_right',
 10: 'right',
 9: 'right',
 8: 'up',
 3: 'up',
 2: 'right',
 1: 'right',
 0: 'right',
 23: 'down',
 22: 'take_whiskr',
 16: 'take_whiskr',
 15: 'up',
 14: 'up',
 54: 'down',
 55: 'down',
 48: 'down',
 40: 'up',
 41: 'right',
 7: 'up',
 6: 'right',
 5: 'up',
 4: 'up',
 46: 'up',
 47: 'up',
 43: 'go_right',
 42: 'right',
 13: 'up',
 12: 'up',
 39: 'up',
 38: 'right',
 37: 'up',
 36: 'up',
 51: 'up',
 50: 'right',
 49: 'up',
 35: 'up',
 34: 'right',
 33: 'right',
 32: 'right',
 21: 'up',
 20: 'up',
 45: 'up',
 44: 'up',
 59: 'left',
 58: 'right',
 57: 'right',
 56: 'right',
 29: 'down',
 28: 'down',
 53: 'up',
 52: 'up',
 30: 'right',
 61: 'down',
 60: 'down',
 31: 'left',
 62: 'cook',
 63: 'left'}

In [20]:
agent.print_policy(agent.easy_policy)

 →  →  →  ←      ↓  ↓  C  ← 
 ↓  ↑  →  ↑      ↑  ↑  ↓  ↓ 
 ↑  →  →  R      ↑  ↑  ↑  ↑ 
 →  →  →  ↑      ↑  ↑  →  ↑ 


 →  →  →  ←      ↓  ↓  →  ← 
 T  ↑  →  ↑      ↑  ↑  T  ↓ 
 ↑  →  →  R      ↑  ↑  ↑  ↑ 
 →  →  →  ↑      ↑  ↑  →  ↑ 


In [21]:
# agent.discount_factor=0.1
# agent.improve_policy(100)

In [22]:
# agent.easy_policy

In [23]:
# agent.print_policy(agent.easy_policy)

In [24]:
# agent.discount_factor=0.9
# agent.improve_policy(100)

In [25]:
# agent.easy_policy

In [26]:
# agent.print_policy(agent.easy_policy)

In [27]:
agent.monte_carlo_online_control(10, 0.5)

ValueError: a must be 1-dimensional or an integer